In [1]:
import numpy as np
import pandas as pd
import csv
from tqdm import tqdm

In [2]:
r = []

with open("/home/rizanb/Documents/hob_pred/data/raw/hob.txt", "r") as f:
    for line in f:
        r_line = line.split(" ")
        r.append(r_line)

r[0:5]

[['1',
  'adrenaline',
  'B',
  '-3.49',
  '-0.90',
  '9.9',
  '4(2),',
  '6,',
  '12',
  '1',
  '1',
  '0.90\n'],
 ['2',
  'alprenolol',
  'B',
  '0.40',
  '-0.89',
  '9.2',
  '6,',
  '8,',
  '9,',
  '10',
  '1',
  '2',
  '2.80\n'],
 ['3', 'clomethiazole', 'N', '2.12', '0.00', '9', '1', '3', '3.86\n'],
 ['4', 'coumarin', 'N', '1.39', '0.00', '7,', '9', '1', '2', '2.92\n'],
 ['5',
  'dobutamine',
  'B',
  '-0.48e',
  '-0.90',
  '9.5',
  '4(3),',
  '9',
  '1',
  '1',
  '0.70\n']]

## keep drug_name, acid/base/neutral category and hob category 

In [3]:
p = []

for i in r:
    e = [i[1], i[2], i[-3]]
    p.append(e)

p[0:3]

[['adrenaline', 'B', '1'],
 ['alprenolol', 'B', '1'],
 ['clomethiazole', 'N', '1']]

## save drug hob values

In [4]:
with open("/home/rizanb/Documents/hob_pred/data/interim/hob.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(p)

In [5]:
%pip install pubchempy

  Using cached pubchempy-1.0.4-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pubchempy as pcp


In [7]:
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski

In [8]:
import numpy as np
import pandas as pd

In [9]:
drug_names = []
for i in p:
    drug_names.append(i[0])
drug_names

['adrenaline',
 'alprenolol',
 'clomethiazole',
 'coumarin',
 'dobutamine',
 'domperidone',
 'dopamine',
 'epanolol',
 'estradiol',
 'felodipine',
 'hydralazine',
 'isradipine',
 'ketamine',
 'lofepramine',
 'lovastatin',
 'mebendazole',
 'meptazinol',
 'mercaptopurine',
 'nabumetone',
 'nalbuphine',
 'naloxone',
 'nimodipine',
 'nisoldipine',
 'nitrendipine',
 'phenolphthalein',
 'probucol',
 'prochlorperazine',
 'progesterone',
 'selegiline',
 'simvastatin',
 'sumatriptan',
 'tacrine',
 'terbutaline',
 'testosterone',
 'tetrahydrocannabinol',
 'venlafaxine',
 'xamoterol',
 'acebutolol',
 'alprazolam',
 'amitriptyline',
 'chlorpromazine',
 'cisapride',
 'clemastin',
 'chlorothiazide',
 'desipramine',
 'dextropropoxyphene',
 'diltiazem',
 'diprafenone',
 'doxepin',
 'encainide',
 'ethinyl',
 'etilefrine',
 'famotidine',
 'fluorouracil',
 'imipramine',
 'indoramine',
 'isoprenaline',
 'isotretinoin',
 'labetalol',
 'lidocaine',
 'lorcainide',
 'medifoxamine',
 'metoprolol',
 'mianserin'

In [10]:
len(drug_names)

232

In [28]:
drugs_with_props = []
c_not_found = []

drugs_with_props.append(["drug_name", "smiles", "hacc", "hdon", "mw", "logp", "mr", "tpsa", "nrot"])

for d in tqdm(drug_names):
    c = pcp.get_compounds(d, "name")

    if not c:
        c_not_found.append(d)
        print(f"compound not found: {d}, skipping")
        continue
    
    c = c[0]
    smiles = c.canonical_smiles
    print(f"{d} smiles: {smiles}") 
    mol = Chem.MolFromSmiles(smiles) 
    hacc = Descriptors.NumHAcceptors(mol)
    hdon = Descriptors.NumHDonors(mol)
    mw = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    mr = Descriptors.MolMR(mol)
    tpsa = Descriptors.TPSA(mol)
    nrot = Descriptors.NumRotatableBonds(mol)

    props_for_d = [d, smiles, hacc, hdon, mw, logp, mr, tpsa, nrot]
    print(props_for_d)
    drugs_with_props.append(props_for_d)

print(drugs_with_props[0:5])

  0%|▎                                                                           | 1/232 [00:01<04:09,  1.08s/it]

adrenaline smiles: CNCC(C1=CC(=C(C=C1)O)O)O
['adrenaline', 'CNCC(C1=CC(=C(C=C1)O)O)O', 4, 4, 183.207, 0.35059999999999986, 48.65810000000003, 72.72, 3]


  1%|▋                                                                           | 2/232 [00:02<04:20,  1.13s/it]

alprenolol smiles: CC(C)NCC(COC1=CC=CC=C1CC=C)O
['alprenolol', 'CC(C)NCC(COC1=CC=CC=C1CC=C)O', 3, 2, 249.35399999999998, 2.1528000000000005, 74.98350000000003, 41.489999999999995, 8]


  1%|▉                                                                           | 3/232 [00:03<04:23,  1.15s/it]

...

valproic smiles: CCCC(CCC)C(=O)O
['valproic', 'CCCC(CCC)C(=O)O', 1, 1, 144.214, 2.2874, 40.9418, 37.3, 5]


100%|█████████████████████████████████████████████████████████████████████████▋| 231/232 [04:50<00:01,  1.17s/it]

warfarin smiles: CC(=O)CC(C1=CC=CC=C1)C2=C(C3=CC=CC=C3OC2=O)O
['warfarin', 'CC(=O)CC(C1=CC=CC=C1)C2=C(C3=CC=CC=C3OC2=O)O', 4, 1, 308.333, 3.6096000000000035, 87.73180000000006, 67.51, 4]


100%|██████████████████████████████████████████████████████████████████████████| 232/232 [04:51<00:00,  1.26s/it]

zalcitabine smiles: C1CC(OC1CO)N2C=CC(=NC2=O)N
['zalcitabine', 'C1CC(OC1CO)N2C=CC(=NC2=O)N', 6, 2, 211.22099999999998, -0.5046000000000004, 53.13920000000002, 90.36999999999999, 2]
[['drug_name', 'smiles', 'hacc', 'hdon', 'mw', 'logp', 'mr', 'tpsa', 'nrot'], ['adrenaline', 'CNCC(C1=CC(=C(C=C1)O)O)O', 4, 4, 183.207, 0.35059999999999986, 48.65810000000003, 72.72, 3], ['alprenolol', 'CC(C)NCC(COC1=CC=CC=C1CC=C)O', 3, 2, 249.35399999999998, 2.1528000000000005, 74.98350000000003, 41.489999999999995, 8], ['clomethiazole', 'CC1=C(SC=N1)CCCl', 2, 0, 161.65699999999998, 2.2328200000000002, 41.27500000000002, 12.89, 2], ['coumarin', 'C1=CC=C2C(=C1)C=CC(=O)O2', 2, 0, 146.14499999999995, 1.7929999999999997, 42.484000000000016, 30.21, 0]]


In [29]:
len(drugs_with_props)

229

In [30]:
drugs_with_props[0:3]

[['drug_name', 'smiles', 'hacc', 'hdon', 'mw', 'logp', 'mr', 'tpsa', 'nrot'],
 ['adrenaline',
  'CNCC(C1=CC(=C(C=C1)O)O)O',
  4,
  4,
  183.207,
  0.35059999999999986,
  48.65810000000003,
  72.72,
  3],
 ['alprenolol',
  'CC(C)NCC(COC1=CC=CC=C1CC=C)O',
  3,
  2,
  249.35399999999998,
  2.1528000000000005,
  74.98350000000003,
  41.489999999999995,
  8]]

In [34]:
c_not_found

['ethinyl', 'acetylsalicylic', 'flecaninide', 'ketolorac']

## save drug properties

In [ ]:
with open("/home/rizanb/Documents/hob_pred/data/interim/drug_props.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(drugs_with_props)

In [18]:
drug_properties_df = pd.read_csv("/home/rizanb/Documents/hob_pred/data/interim/drug_props.csv")
drug_properties_df.head()

,drug_name,smiles,hacc,hdon,mw,logp,mr,tpsa,nrot
0,adrenaline,CNCC(C1=CC(=C(C=C1)O)O)O,4,4,183.207,0.35060,48.6581,72.72,3
1,alprenolol,CC(C)NCC(COC1=CC=CC=C1CC=C)O,3,2,249.354,2.15280,74.9835,41.49,8
2,clomethiazole,CC1=C(SC=N1)CCCl,2,0,161.657,2.23282,41.2750,12.89,2
3,coumarin,C1=CC=C2C(=C1)C=CC(=O)O2,2,0,146.145,1.79300,42.4840,30.21,0
4,dobutamine,CC(CCC1=CC=C(C=C1)O)NCCC2=CC(=C(C=C2)O)O,4,4,301.386,2.95680,87.3681,72.72,7


In [19]:
drug_hob_df = pd.read_csv("/home/rizanb/Documents/hob_pred/data/interim/hob.csv")
drug_hob_df.head()

,drug_name,type,hob_category
0,adrenaline,B,1
1,alprenolol,B,1
2,clomethiazole,N,1
3,coumarin,N,1
4,dobutamine,B,1


## merge properties and hob datasets

In [20]:
merged = pd.merge(drug_properties_df, drug_hob_df, on="drug_name")
merged.head()

,drug_name,smiles,hacc,hdon,mw,logp,mr,tpsa,nrot,type,hob_category
0,adrenaline,CNCC(C1=CC(=C(C=C1)O)O)O,4,4,183.207,0.35060,48.6581,72.72,3,B,1
1,alprenolol,CC(C)NCC(COC1=CC=CC=C1CC=C)O,3,2,249.354,2.15280,74.9835,41.49,8,B,1
2,clomethiazole,CC1=C(SC=N1)CCCl,2,0,161.657,2.23282,41.2750,12.89,2,N,1
3,coumarin,C1=CC=C2C(=C1)C=CC(=O)O2,2,0,146.145,1.79300,42.4840,30.21,0,N,1
4,dobutamine,CC(CCC1=CC=C(C=C1)O)NCCC2=CC(=C(C=C2)O)O,4,4,301.386,2.95680,87.3681,72.72,7,B,1


In [21]:
merged.describe()

,hacc,hdon,mw,logp,mr,tpsa,nrot,hob_category
count,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000,230.000000
mean,3.773913,1.491304,295.845565,2.464500,81.385327,59.876130,4.095652,2.786957
std,1.695413,1.084795,83.482243,1.662684,23.024807,30.645149,2.829890,1.082938
min,1.000000,0.000000,46.069000,-1.494200,12.759800,3.240000,0.000000,1.000000
25%,2.000000,1.000000,249.565500,1.354455,67.826125,39.745000,2.000000,2.000000
50%,4.000000,1.000000,295.941000,2.396050,82.683500,58.560000,4.000000,3.000000
75%,5.000000,2.000000,342.775500,3.598630,95.210700,75.675000,6.000000,4.000000
max,9.000000,4.000000,645.319000,9.908300,156.806600,175.830000,13.000000,4.000000


In [15]:
merged.isna().sum()

drug_name        0
smiles           0
hacc             0
hdon             0
mw               0
logp             0
mr               0
tpsa             0
nrot             0
 type            0
 hob_category    0
dtype: int64

In [22]:
merged.isnull().sum()

drug_name        0
smiles           0
hacc             0
hdon             0
mw               0
logp             0
mr               0
tpsa             0
nrot             0
 type            0
 hob_category    0
dtype: int64

In [23]:
merged.to_csv("/home/rizanb/Documents/hob_pred/data/interim/drug_properties_hob.csv", index=None)

In [24]:
merged.head()

,drug_name,smiles,hacc,hdon,mw,logp,mr,tpsa,nrot,type,hob_category
0,adrenaline,CNCC(C1=CC(=C(C=C1)O)O)O,4,4,183.207,0.35060,48.6581,72.72,3,B,1
1,alprenolol,CC(C)NCC(COC1=CC=CC=C1CC=C)O,3,2,249.354,2.15280,74.9835,41.49,8,B,1
2,clomethiazole,CC1=C(SC=N1)CCCl,2,0,161.657,2.23282,41.2750,12.89,2,N,1
3,coumarin,C1=CC=C2C(=C1)C=CC(=O)O2,2,0,146.145,1.79300,42.4840,30.21,0,N,1
4,dobutamine,CC(CCC1=CC=C(C=C1)O)NCCC2=CC(=C(C=C2)O)O,4,4,301.386,2.95680,87.3681,72.72,7,B,1
